In [6]:
import json
utterance_file = "utterance1.txt"
dialog_cnt = 1000
outfile_cnt = 10
json_list = []
other_domains = ["taxi", "train", "bus", "police", "hotel", "attraction", "hospital"]
service_domain = "restaurant"
turn_id = 0

In [7]:
with open(utterance_file) as fp:
    Lines = fp.readlines()

In [8]:
def init_dict(dialog_cnt):
    turn_id = 0
    dialog_cnt += 1
    dialogue_id = "CAiFE" + str(dialog_cnt)
    line_dict = {"dialogue_id": dialogue_id, "services": ["restaurant"], 
                 "turns": [{"frames": []}]}
    return line_dict

In [9]:
def add_other_domains():
    d_list = []
    for domain in other_domains:
        d = {"actions": [], "service": domain, "slots": [], 
            "state": {"active_intent": "NONE", "requested_slots": [], "slot_values": {}}}
        d_list.append(d)
    return d_list

In [10]:
for line in Lines:
    if line[0:4] == "----":
        line_dict = init_dict(dialog_cnt)
        continue
    
    # identify the speaker using ':' as separator and taking letter up to ':'
    ind = line.index(':')
    speaker = line[0: ind]
    
    #identify if keyword exists in a line and get start and end indices
    ind1 = line.index('[')
    ind2 = line.index(']')
    
    if speaker == "SYSTEM":
        if ind2 - ind1 == 1: # no keyword exists
            frame_dict = []
        else: # slot name, value exists
            line_lowercase = line.lower() # get the lower case versio of the line
            slot_name_val = line_lowercase[ind1+1: ind2] # extract slot name and value pair
            name_list = slot_name_val.split(',')
            slotName = name_list[0].strip()
            slotVal = name_list[1].strip()
            
            start = line_lowercase.index(slotVal) # get the start index of value
            exclusive_end = start + len(slotVal) # get the end index of value
            frame_dict = {"actions": [], "service": service_domain,
                          "slots": [{"exclusive_end": exclusive_end,
                                     "slot": slotName,
                                     "start": start,
                                     "value": slotVal
                                    }
                                   ]
                         }
        line_dict["turns"][0]["frames"].append(frame_dict)
        
    else: # case of speake being USER
        line_lowercase = line.lower() # get the lower case versio of the line
        slot_name_val = line_lowercase[ind1+1: ind2] # extract slot name and value pair
        name_list = slot_name_val.split(',')
        slotName = name_list[0].strip() # 1st value is slot name
        slotVal = name_list[1].strip()  # 2nd value is the slot value
        active_intent = name_list[2].strip() # 3rd is the active intent
        
        start = line_lowercase.index(slotVal) # get the start index of value
        exclusive_end = start + len(slotVal) # get the end index of value
        frame_dict = {"actions": [], "service": service_domain,
                      "slots": [{"exclusive_end": exclusive_end,
                                 "slot": slotName,
                                 "start": start,
                                 "value": slotVal
                                }
                               ],
                      "state": {"active_intent": active_intent,
                                "requested_slots": [],
                                "slot_values": {}
                               }
                      
                     }
        line_dict["turns"][0]["frames"].append(frame_dict)
        other_domain_list = add_other_domains() # get values for other domains
        for val in other_domain_list:
            line_dict["turns"][0]["frames"].append(val)
        
    utterance = line[ind2+3: -3]
    
    line_dict["turns"][0]["frames"].append({"speaker": speaker})
    line_dict["turns"][0]["frames"].append({"turn_id": turn_id})
    turn_id += 1
    line_dict["turns"][0]["frames"].append({"utterance": utterance})
    json_list.append(line_dict)

In [11]:
json_object = json.dumps(json_list, indent=2)
outfile_cnt += 1
json_file_name = "dialogues_0" + str(outfile_cnt) + ".json"
with open(json_file_name, "w") as fp:
    fp.write(json_object)